<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Managing Virtual Machines with Python

## Azure Actions Covered

* Listing virtual machines, subnets, virtual nets, disks, and public IP addresses
* Powering on and powering off virtual machines
* Deleting virtual machines, subnets, virtual nets, disks, and public IP addresses

In this lecture, we're going to take a look at how to manage virtual machines via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credential as well as our network and compute management clients.

In [2]:
credential = AzureCliCredential()
compute_client = ComputeManagementClient(credential, AZURE_SUBSCRIPTION_ID)
network_client = NetworkManagementClient(credential, AZURE_SUBSCRIPTION_ID)

## Virtual Machines

Let's list all of our virtual machines, regardless of resource group.

In [4]:
for vm in compute_client.virtual_machines.list_all():
    print(vm.name)

test-vm


We can also list virtual machines by specific resource group.

In [5]:
for vm in compute_client.virtual_machines.list(DEFAULT_RESOURCE_GROUP):
    print(vm.name)

test-vm


We can power down virtual machines when they're not in use using `begin_power_off()`.

In [7]:
poller = compute_client.virtual_machines.begin_power_off(
    resource_group=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)

To check the current status of our VM, can use the `instance_view()` method.

In [18]:
vm = compute_client.virtual_machines.instance_view(
    resource_group_name=DEFAULT_RESOURCE_GROUP, 
    vm_name='test-vm'
)
print([status.display_status for status in vm.statuses])

To start the VM back up, we can use `begin_start()`.

In [23]:
poller = compute_client.virtual_machines.begin_start(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)

Let's check to make sure the VM is running again.

In [26]:
vm = compute_client.virtual_machines.instance_view(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)
print([status.display_status for status in vm.statuses])

Instead of stopping and starting the VM, we can also just call the `begin_restart()` method.

In [29]:
poller = compute_client.virtual_machines.begin_restart(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)

Finally, we can delete our virtual machine.

In [33]:
poller = compute_client.virtual_machines.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    vm_name='test-vm'
)
result = poller.result()

## Network Interfaces

We can list all network interface clients or list them by resource group, like we did with our virtual machines.

In [35]:
for nic in network_client.network_interfaces.list_all():
    print(nic.name)

test-nic


In [36]:
for nic in network_client.network_interfaces.list(DEFAULT_RESOURCE_GROUP):
    print(nic.name)

test-nic


We can delete our NIC as well, since we no longer have a VM.

In [37]:
poller = network_client.network_interfaces.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    network_interface_name='test-nic'
)
result = poller.result()

## Public IP Addresses

Much of the same functionality is available for public IP addresses that we've created as well.

In [39]:
for ip in network_client.public_ip_addresses.list_all():
    print(ip.name)

vm-test-ip


In [40]:
poller = network_client.public_ip_addresses.begin_delete(
    DEFAULT_RESOURCE_GROUP,
    'vm-test-ip'
)

## Subnets

We can list and then delete our subnet on our virtual network.

In [42]:
for s in network_client.subnets.list(DEFAULT_RESOURCE_GROUP, 'test-vnet'):
    print(s.name)

test-subnet


In [45]:
poller = network_client.subnets.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    virtual_network_name='test-vnet',
    subnet_name='test-subnet'
)

In [46]:
for s in network_client.subnets.list(DEFAULT_RESOURCE_GROUP, 'test-vnet'):
    print(s.name)

## Virtual Networks

Finally, we interact with our virtual networks by listing or deleting them as well.

In [43]:
for network in network_client.virtual_networks.list_all():
    print(network.name)

test-vnet


In [47]:
poller = network_client.virtual_networks.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    virtual_network_name='test-vnet'
)

## Disks

In Azure, disks may be created and attached to your virtual machine when it's created. We can list and delete these to fully clean up our environment.

In [48]:
for disk in compute_client.disks.list_by_resource_group(DEFAULT_RESOURCE_GROUP):
    print(disk.name)

test-vm_OsDisk_1_14f4c7da54df4c8195165e95fda8f5c8


In [49]:
poller = compute_client.disks.begin_delete(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    disk_name='test-vm_OsDisk_1_14f4c7da54df4c8195165e95fda8f5c8'
)